In [2]:
name = "kdd-cup-2014-predicting-excitement-at-donors-choose"

In [3]:
import pandas as pd
# loading all dataframes
donations_df = pd.read_csv(f'datasets_kaggle/{name}/donations.csv')
essays_df = pd.read_csv(f'datasets_kaggle/{name}/essays.csv')
outcomes_df = pd.read_csv(f'datasets_kaggle/{name}/outcomes.csv')
projects_df = pd.read_csv(f'datasets_kaggle/{name}/projects.csv')
resources_df = pd.read_csv(f'datasets_kaggle/{name}/resources.csv')
sample_submission_df = pd.read_csv(f'datasets_kaggle/{name}/sampleSubmission.csv')

# Applying Jimmy's Code

In [4]:
# Joining projects with outcomes
# Usefulness: This forms the base dataframe where we can add more features that will help in predicting the "is_exciting" label.
# Input samples: 'projectid' from projects_df: ['dd291f97dc6b32b7c9f052794efc4929', '673d26170aeaf573d6e4c7565a65915b', '8f0e3bc89dc5304f71f167201242fa5e']
#                'projectid' from outcomes_df: ['c6bc4035d501cc99ab78072d8544e3e4', 'cc7e0ca72280d0b08846ec7f15549406', 'a75a074419bcd15f3c22a8dcddd5f7cb']
merged_df = pd.merge(projects_df, outcomes_df, on='projectid', how='inner')

In [6]:
merged_df

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,at_least_1_green_donation,great_chat,three_or_more_non_teacher_referred_donors,one_non_teacher_referred_donor_giving_100_plus,donation_from_thoughtful_donor,great_messages_proportion,teacher_referred_count,non_teacher_referred_count,total_price_with_fulfillment,students_per_dollar
0,62526d85d2a1818432d03d600969e99c,ebc7c90b6c92a069432e0714b8d93dfd,5aca9711ff0e4b37db48701f46f73036,1.713710e+11,41.972419,-88.174597,Bartlett,IL,60103.0,suburban,...,f,t,t,t,f,80.0,0.0,6.0,474.36,0.014757
1,33d59ac771b80222ad63ef0f4ac47ade,de83b4c1f6428a15032c207c1d5e572a,d91a805b213bf74ae77b94e0de2b73ad,1.601530e+11,43.501154,-112.056780,Idaho Falls,ID,83402.0,urban,...,NaN,f,NaN,NaN,NaN,NaN,NaN,NaN,263.24,0.113964
2,1a3aaeffc56dd2a421e37d8298024c0a,f4c9ed095b85458dcf858e25f203af00,9310d3eb447a4e46bc5fc31ed007ceac,3.302610e+11,42.888244,-71.320224,Derry,NH,3038.0,suburban,...,t,f,f,f,f,NaN,0.0,2.0,315.09,0.729950
3,33aa19ee4da4c5adf47d0dfb84fab5ef,17768031eb40de8d4497dbb54df48742,9ac70da58322783f82152eecc140a812,5.103240e+11,37.476158,-77.488397,Richmond,VA,23224.0,urban,...,t,f,f,f,f,NaN,0.0,1.0,262.94,0.068457
4,e31c0ea8b68f404699dfb0d39e9bc99b,0f1bc5b4700fd33383be104442660178,cb9f688cf59e3ee22a087d616ca8f5d7,1.709930e+11,41.952851,-87.650233,Chicago,IL,60613.0,urban,...,t,f,f,t,f,50.0,0.0,2.0,543.41,0.128816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619321,a7236ea96c812895cafc5d700d779147,c4f90a541cb81e68cd7ad180309985ae,a1e14d253e77ffccc074705527845b4a,3.600081e+11,40.809677,-73.939178,New York,NY,10037.0,urban,...,t,f,f,t,f,NaN,0.0,1.0,NaN,NaN
619322,e02da37beb332eb66c2d2ba989c597ad,d95c94eea994274d614be50bddd89973,bbd70e92262f0f0a342df5d1f3119d8e,3.600097e+11,40.701778,-74.011840,New York,NY,10004.0,urban,...,t,f,f,t,f,NaN,0.0,1.0,NaN,NaN
619323,82e536f14eadf2671a70e03416f695a3,b00d2790934ce7487c99bc9d6e1d5d4d,0c5b1a4b6985dcd42c72b732c774474c,3.600077e+11,40.770233,-73.950760,New York,NY,10075.0,urban,...,t,f,f,t,f,NaN,0.0,1.0,NaN,NaN
619324,e139df754a873a62d93daa56acbf8040,e5476c2ef97b33614445a17554535adc,a81c26262a3ad70702a8f14e5329b023,3.600077e+11,40.765517,-73.960090,New York,NY,10065.0,NaN,...,t,f,f,t,f,NaN,0.0,1.0,NaN,NaN


In [7]:

#### Feature 1: `total_price_with_fulfillment`
#Calculate the total price of a project including the `fulfillment_labor_materials`.

# Feature: Total price of the project including fulfillment labor materials
# Usefulness: The total financial burden of a project, including hidden costs, may influence its chances of being considered "exciting."
# Input samples: 'total_price_excluding_optional_support': [2148.49, 227.37, 188.80], 'fulfillment_labor_materials': [30.0, 35.0, 35.0]
merged_df['total_price_with_fulfillment'] = merged_df['total_price_excluding_optional_support'] + merged_df['fulfillment_labor_materials']

#### Feature 2: `students_per_dollar`
#Calculate the number of students reached per dollar spent on the project.

# Feature: Number of students reached per dollar
# Usefulness: Projects that impact more students per dollar may be more "exciting" due to their cost-effectiveness.
# Input samples: 'students_reached': [125.0, 62.0, 80.0], 'total_price_with_fulfillment': [2178.49, 262.37, 223.80]
merged_df['students_per_dollar'] = merged_df['students_reached'] / merged_df['total_price_with_fulfillment']

#### Drop Columns
#We will drop the columns that are already transformed or do not add much information to the predictive model.

# Explanation: Dropping columns that have been transformed or are not directly useful for predicting 'is_exciting'
merged_df.drop(columns=['total_price_excluding_optional_support', 'fulfillment_labor_materials'], inplace=True)

In [8]:
# Joining merged_df with donations_df on projectid
# Usefulness: This allows us to add features based on the donations each project received, which could be a strong indicator of whether a project is "exciting."
# Input samples: 'projectid' from merged_df: ['195d03cc23a38b2990dff4b81e774f28', '3c884a0ee29b7dea836b887d60d79c08', 'ce4c4ef12a619490c211f9534797796b']
#                'projectid' from donations_df: ['54622a1620e0047229ab1a466f2092dd', 'be8d90d9c4c8f4cbe8e75b39a835f46a', 'b5a6a8c2df3137d3be7f1b0d03f7ffed']
merged_with_donations_df = pd.merge(merged_df, donations_df, on='projectid', how='left')

#### Feature 3: `average_donation_amount`
#Calculate the average donation amount received by each project.

# Feature: Average donation amount for each project
# Usefulness: A higher average donation amount could indicate that donors find the project more compelling or "exciting."
# Input samples: 'donation_total': [17.65, 25.00, 100.00]
merged_with_donations_df['average_donation_amount'] = merged_with_donations_df.groupby('projectid')['donation_total'].transform('mean')

#### Feature 4: `donor_teacher_ratio`
# Calculate the ratio of teacher to non-teacher donors for each project.

# Feature: Ratio of teacher donors to non-teacher donors for each project
# Usefulness: A higher ratio of teacher donors could indicate that the project is more "exciting" to educators, which may be a good sign.
# Input samples: 'is_teacher_acct': ['t', 'f', 'f']
merged_with_donations_df['is_teacher'] = merged_with_donations_df['is_teacher_acct'].apply(lambda x: 1 if x == 't' else 0)
merged_with_donations_df['teacher_donor_ratio'] = merged_with_donations_df.groupby('projectid')['is_teacher'].transform('mean')

#### Drop Columns
# We'll drop columns that have been used for aggregations or transformations and won't be directly useful for the model.

# Explanation: Dropping columns that have been used for aggregation or transformation
merged_with_donations_df.drop(columns=['donation_total', 'is_teacher_acct', 'is_teacher'], inplace=True)

In [11]:
merged_with_donations_df.describe()

,school_ncesid,school_latitude,school_longitude,school_zip,total_price_including_optional_support,students_reached,is_exciting,great_messages_proportion,teacher_referred_count,non_teacher_referred_count,total_price_with_fulfillment,students_per_dollar,donor_zip,donation_to_project,donation_optional_support,average_donation_amount,teacher_donor_ratio
count,6.261300e+04,67076.000000,67076.000000,67076.000000,6.707600e+04,67066.000000,67076.000000,55155.000000,61195.000000,61195.000000,64738.000000,64728.000000,21602.000000,48483.000000,48483.000000,48483.000000,67076.000000
mean,2.557701e+11,37.187369,-94.360712,54169.825452,1.381381e+03,92.891868,0.108608,53.870166,1.787597,9.539505,609.771172,0.210185,51199.365568,65.839890,10.409881,76.249771,0.085783
std,1.623766e+11,4.781100,18.213328,31581.476464,2.902883e+04,166.633026,0.311149,31.308943,4.722826,13.956722,1065.627774,0.404640,33301.982044,255.531649,27.068378,234.161226,0.207913
min,1.000050e+10,18.249140,-164.792083,1020.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,44.680000,0.000188,0.000000,0.000000,0.000000,0.220000,0.000000
25%,8.033600e+10,34.026493,-117.089933,27832.000000,3.553500e+02,22.000000,0.000000,33.000000,0.000000,2.000000,330.050000,0.044922,20141.000000,9.350000,1.500000,18.333333,0.000000
50%,2.502790e+11,37.462179,-87.785355,54301.000000,5.159500e+02,30.000000,0.000000,54.000000,0.000000,5.000000,464.850000,0.083243,48047.000000,21.250000,3.750000,32.690000,0.000000
75%,3.704720e+11,40.758002,-79.825195,90003.000000,7.658600e+02,96.000000,0.000000,77.000000,2.000000,11.000000,675.920000,0.194455,90016.000000,50.000000,7.500000,69.496667,0.000000
max,6.100003e+11,67.258157,-66.628036,99827.000000,1.219512e+06,15000.000000,1.000000,100.000000,83.000000,161.000000,139755.410000,20.256310,99775.000000,33475.410000,1599.820000,33475.410000,1.000000


In [27]:
merged_with_donations_df.isna().sum(axis=1)

0         4
1        28
2         4
3        22
4         0
         ..
67071     2
67072     4
67073     2
67074     2
67075    21
Length: 67076, dtype: int64

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Assume merged_with_donations_sampled_df is the final DataFrame after feature engineering
# Convert 'is_exciting' to numerical labels
label_encoder = LabelEncoder()
merged_with_donations_df['is_exciting'] = label_encoder.fit_transform(merged_with_donations_df['is_exciting'])

# Select features and target variable
features = ['total_price_with_fulfillment', 'students_per_dollar', 'average_donation_amount', 'teacher_donor_ratio']
X = merged_with_donations_df[features]
y = merged_with_donations_df['is_exciting']

# Drop rows with missing values in feature columns
X_dropped = X.dropna()
# Drop corresponding rows in target variable
y_dropped = y.loc[X_dropped.index]


In [10]:
X_dropped.isna().sum(axis=1)
len(X_dropped)

3042874

In [11]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

KeyboardInterrupt: 

In [48]:
X_dropped.columns

Index(['total_price_with_fulfillment', 'students_per_dollar',
       'average_donation_amount', 'teacher_donor_ratio'],
      dtype='object')

# Without FE just main table with feature selection

In [35]:
merged_df_new = pd.merge(projects_df, outcomes_df, on='projectid', how='inner')

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# Assuming merged_df is your main DataFrame after merging projects_df and outcomes_df
# Convert 'is_exciting' to numerical labels for machine learning
label_encoder = LabelEncoder()
merged_df_new['is_exciting'] = label_encoder.fit_transform(merged_df_new['is_exciting'].fillna(''))

# Select some existing features (before feature engineering) for demonstration
# We'll consider 'students_reached' and 'total_price_excluding_optional_support' for this example
features = ['students_reached', 'total_price_excluding_optional_support']

# Drop rows with missing values in feature columns
X_dropped = merged_df_new[features].dropna()

# Drop corresponding rows in target variable
y_dropped = merged_df_new['is_exciting'].loc[X_dropped.index]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.90465746355309


# Without FE just main table without feature selection

In [42]:
merged_df_new_new = pd.merge(projects_df, outcomes_df, on='projectid', how='inner')

In [44]:
# Assuming merged_df is your main DataFrame after merging projects_df and outcomes_df
# Convert 'is_exciting' to numerical labels for machine learning
label_encoder = LabelEncoder()
merged_df_new_new['is_exciting'] = label_encoder.fit_transform(merged_df_new_new['is_exciting'].fillna(''))

# Drop rows with missing values across all columns
clean_df = merged_df_new_new.dropna()

# Filter only numerical features
numerical_features = clean_df.select_dtypes(include=['float64', 'int64']).columns

# Separate features and target variable
X_dropped = clean_df[numerical_features]  # Keep only numerical features
X_dropped = X_dropped.drop(columns=['is_exciting'])
y_dropped = clean_df['is_exciting']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9824506424318395


In [45]:
numerical_features

Index(['Unnamed: 0_x', 'school_ncesid', 'school_latitude', 'school_longitude',
       'school_zip', 'fulfillment_labor_materials',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support', 'students_reached',
       'Unnamed: 0_y', 'is_exciting', 'great_messages_proportion',
       'teacher_referred_count', 'non_teacher_referred_count'],
      dtype='object')